In [1]:
import os
import logging
from argparse import ArgumentParser

colmap_command = "colmap"
no_gpu = False
skip_matching = False
source_path = '../../example_data'
camera = "PINHOLE"

use_gpu = 1 if not no_gpu else 0

if not skip_matching:
    os.makedirs(source_path + "/distorted/sparse", exist_ok=True)

    ## Feature extraction
    feat_extracton_cmd = colmap_command + " feature_extractor "\
        "--database_path " + source_path + "/distorted/database.db \
        --image_path " + source_path + "/images \
        --ImageReader.single_camera 1 \
        --ImageReader.camera_model " + camera + " \
        --SiftExtraction.use_gpu " + str(use_gpu)
    exit_code = os.system(feat_extracton_cmd)
    if exit_code != 0:
        logging.error(f"Feature extraction failed with code {exit_code}. Exiting.")
        exit(exit_code)

    ## Feature matching
    feat_matching_cmd = colmap_command + " exhaustive_matcher \
        --database_path " + source_path + "/distorted/database.db \
        --SiftMatching.use_gpu " + str(use_gpu)
    exit_code = os.system(feat_matching_cmd)
    if exit_code != 0:
        logging.error(f"Feature matching failed with code {exit_code}. Exiting.")
        exit(exit_code)

    ### Bundle adjustment
    # The default Mapper tolerance is unnecessarily large,
    # decreasing it speeds up bundle adjustment steps.
    mapper_cmd = (colmap_command + " mapper \
        --database_path " + source_path + "/distorted/database.db \
        --image_path "  + source_path + "/images \
        --output_path "  + source_path + "/distorted/sparse \
        --Mapper.ba_global_function_tolerance=0.000001")
    exit_code = os.system(mapper_cmd)
    if exit_code != 0:
        logging.error(f"Mapper failed with code {exit_code}. Exiting.")
        exit(exit_code)

    ### Convert the model to a text file
    os.makedirs(source_path + "/sparse", exist_ok=True)
    convert_cmd = colmap_command + " model_converter \
        --input_path " + source_path + "/distorted/sparse/0 \
        --output_path " + source_path + "/sparse \
        --output_type TXT"
    exit_code = os.system(convert_cmd)
    if exit_code != 0:
        logging.error(f"Model conversion failed with code {exit_code}. Exiting.")
        exit(exit_code)

    ### Delete the distorted folder
    os.system("rm -r " + source_path + "/distorted")

I20241015 07:20:12.052548 106817 misc.cc:198] 
Feature extraction
I20241015 07:20:12.092305 106842 sift.cc:716] Creating SIFT GPU feature extractor
I20241015 07:20:12.195067 106843 feature_extraction.cc:257] Processed file [1/10]
I20241015 07:20:12.195111 106843 feature_extraction.cc:260]   Name:            0.png
I20241015 07:20:12.195120 106843 feature_extraction.cc:286]   Dimensions:      1280 x 720
I20241015 07:20:12.195127 106843 feature_extraction.cc:289]   Camera:          #1 - PINHOLE
I20241015 07:20:12.195133 106843 feature_extraction.cc:292]   Focal Length:    1536.00px
I20241015 07:20:12.195145 106843 feature_extraction.cc:296]   Features:        2249
I20241015 07:20:12.543146 106843 feature_extraction.cc:257] Processed file [2/10]
I20241015 07:20:12.543226 106843 feature_extraction.cc:260]   Name:            1.png
I20241015 07:20:12.543241 106843 feature_extraction.cc:286]   Dimensions:      1280 x 720
I20241015 07:20:12.543254 106843 feature_extraction.cc:289]   Camera:    

In [2]:
# pycolmap = 0.4.0
from aruco_estimator.aruco_scale_factor import ArucoScaleFactor
from colmap_wrapper.colmap import COLMAP
aruco_size = 0.186 # the size of the aruco marker in meters

# Load Colmap project folder
project = COLMAP(project_path=source_path)

# Init & run pose estimation of corners in 3D & estimate mean L2 distance between the four aruco corners
aruco_scale_factor = ArucoScaleFactor(photogrammetry_software=project, aruco_size=aruco_size)
aruco_distance, aruco_corners_3d = aruco_scale_factor.run()
print('Size of the unscaled aruco markers: ', aruco_distance)

# Calculate scaling factor, apply it to the scene and save scaled point cloud
dense, scale_factor = aruco_scale_factor.apply() 
print('Point cloud and poses are scaled by: ', scale_factor)
print('Size of the scaled (true to scale) aruco markers in meters: ', aruco_distance * scale_factor)

# Write Data
aruco_scale_factor.write_data()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D WARNING] Read PLY failed: unable to open file: ../../example_data/fused.ply
Num process:  12


RPly: Unable to open file
100%|██████████| 10/10 [00:00<00:00, 54.64it/s]


Size of the unscaled aruco markers:  2.1843546796395974
Point cloud and poses are scaled by:  0.08515100671777746
Size of the scaled (true to scale) aruco markers in meters:  0.186
